In [2]:
import os
from llama_cpp import Llama

In [3]:
os.getcwd()

'/mnt/c/Users/hew7/Documents/Git/local_rag_workshop/dev'

In [4]:
llm = Llama(model_path = 'Llama-3.2-1B-Instruct-Q6_K.gguf', n_ctx=2048) #set context window HERE!

llama_model_loader: loaded meta data with 35 key-value pairs and 147 tensors from Llama-3.2-1B-Instruct-Q6_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.2 1B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.2
llama_model_loader: - kv   5:                         general.size_label str              = 1B
llama_model_loader: - kv   6:                            general.license str              = llama3.2
llama_model_loader: - kv   7:                       

In [5]:
messages = [
          {"role": "system", "content": "You are an assistant who helps with everyday complex tasks."},
          {
              "role": "user",
              "content": """
              Hello how's it going?
              """
          }
      ]

In [6]:
output = llm.create_chat_completion(
        messages = messages,  
    # "Do you think I'm a smart little boy? ", # Prompt
      # max_tokens=128, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      # stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      # echo=True # Echo the prompt back in the output
) # Generate a completion, can also call create_completion

llama_perf_context_print:        load time =    1630.54 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    52 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    30 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    2910.75 ms /    82 tokens


In [7]:
output["choices"][0]['message']

{'role': 'assistant',
 'content': "I'm doing well, thanks for asking. I'm here to help with any questions or tasks you may have. How can I assist you today?"}

In [8]:
output

{'id': 'chatcmpl-0d012fe2-a4e3-4375-af8b-c23d755289c3',
 'object': 'chat.completion',
 'created': 1733894219,
 'model': 'Llama-3.2-1B-Instruct-Q6_K.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': "I'm doing well, thanks for asking. I'm here to help with any questions or tasks you may have. How can I assist you today?"},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 52, 'completion_tokens': 30, 'total_tokens': 82}}

In [9]:
# seems to only need 5gb extra ram to run with abt 2.5k tokens. Takes about 2 minutes to generate

In [10]:
def llama_chat_gen(input_chat: list[dict]):
    init_output = llm.create_chat_completion(messages = input_chat)
    response = init_output["choices"][0]['message']
    print(response['content'])
    input_chat.append(response)
    return input_chat

In [11]:
messages = llama_chat_gen(messages)

Llama.generate: 51 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    1630.54 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    31 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =    1474.90 ms /    32 tokens


I'm doing well, thanks for asking. I'm here to help with any questions or tasks you may have. How can I assist you today?


In [15]:
messages[-1]['content']

"I'm doing well, thanks for asking. I'm here to help with any questions or tasks you may have. How can I assist you today?"

In [32]:
def llama_chat_func(input_chat, textinput):
    input_chat.append(
        {'role': 'user',
        'content': textinput}
    )
    response = llama_chat_gen(input_chat)
    return response

In [35]:
messages = llama_chat_func(messages, "whats 2+2")

Llama.generate: 105 prefix-match hit, remaining 16 prompt tokens to eval
llama_perf_context_print:        load time =    1216.51 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    16 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     7 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     699.40 ms /    23 tokens


2 + 2 = 4


In [36]:
messages

[{'role': 'system',
  'content': 'You are an assistant who helps with everyday complex tasks.'},
 {'role': 'user',
  'content': "\n              Hello how's it going?\n              "},
 {'role': 'assistant',
  'content': "I'm doing well, thanks for asking. I'm here to help with any questions or tasks you may have. How can I assist you today?"},
 {'role': 'user', 'content': 'whats 2+2'},
 {'role': 'assistant', 'content': '2 + 2 = 4'},
 {'role': 'user', 'content': 'whats 2+2'},
 {'role': 'assistant', 'content': '2 + 2 = 4'}]

In [39]:
class RAGChat():
    ### TO ADD: retry generation func????? Maybe deletion as well????
    def __init__(self):
        self.messages = []
        self.RAGmessages = []
        self.inject_template = f'{INJECT_TEXT}, {USER_MESSAGE}'
        self.chathead = []

    def __call__(self):
        "default call returns non-enhanced chat history"
        return self.messages

    def RAG_chathist(self):
        "returns RAG'd chat history"
        return self.RAGmessages

    def __len__(self):
        return len(self.messages)

    def add_user_message(self, message):
        "add user message. For user input."
        self.messages.append({
                'role': 'user',
                'content': message
            })  

    def add_augmented_user_message(self, injection: list[str], message):
        injection_str = ' '.join(injection)
        self.RAGmessages.append({
                'role': 'user',
                'content': self.inject_template.format(INJECT_TEXT=injection_str, USER_MESSAGE=message)
            })  
        
    def add_system_message(self, message: dict):
        """add system message. For adding to message chains from chatbot generated content
        
        NOTE: ONLY appends the  
        """
        self.messages.append({
                'role': 'system',
                'content': message
            })
        self.RAGmessages.append({
                'role': 'system',
                'content': message
            })

    def temp_chathead(self, chathead, augmented: bool = False):
        "for textgen-only APIs that might not have an inbuilt chathead. Call this instead of default call for temporary chathead."
        self.chathead = [{
            'role': 'system',
            'content': chathead
        }]
        if augmented:
            self.chathead.extend(self.RAGmessages)
        else:
            self.chathead.extend(self.messages)
        return self.chathead
